In [1]:
import numpy as np
import sys, os
sys.path.append(os.path.abspath(".."))

from models.bayesflow_model import BayesFlowModel

f:\sbi_task5\venv\lib\site-packages\bayesflow\trainers.py:27: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [16]:
from tensorflow.keras.models import load_model
from bayesflow.networks import InvertibleNetwork
from bayesflow.amortizers import AmortizedPosterior
import tensorflow as tf

# Load saved summary network
summary_net = load_model("../saved_models/summary_net.keras")

# Reconstruct and warm up posterior network
posterior_net = InvertibleNetwork(num_params=50)
latent = tf.random.normal((2, 50))
condition = tf.random.normal((2, 50))
posterior_net(latent, condition)  # important: triggers variable creation

# Load weights after warmup
posterior_net.load_weights("../saved_models/posterior_net.weights.h5")

# Wrap everything into an amortizer
amortizer = AmortizedPosterior(posterior_net, summary_net)


In [22]:
from scripts.train_bayesflow import prior

# Get one test input (shape: [1, 50])
test_input = prior(1)['prior_draws']
print("Test Input:", test_input)
print("Type:", type(test_input))
print("Shape:", getattr(test_input, "shape", None))


Test Input: [[1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.
  0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1.
  1. 0.]]
Type: <class 'numpy.ndarray'>
Shape: (1, 50)


In [24]:
summary_net.summary()


Model: "summary_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sim_data (InputLayer)       [(None, 50)]              0         
                                                                 
 dense (Dense)               (None, 100)               5100      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
Total params: 10150 (39.65 KB)
Trainable params: 10150 (39.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
from tensorflow.keras.models import load_model
from bayesflow.networks import InvertibleNetwork
from bayesflow.amortizers import AmortizedPosterior
import tensorflow as tf
import numpy as np

# Load saved models
summary_net = load_model("../saved_models/summary_net.keras")
posterior_net = InvertibleNetwork(num_params=50)
latent = tf.random.normal((2, 50))
condition = tf.random.normal((2, 50))
posterior_net(latent, condition)
posterior_net.load_weights("../saved_models/posterior_net.weights.h5")

# Build amortizer
amortizer = AmortizedPosterior(posterior_net, summary_net)

# Safe test input
test_input = np.random.rand(1, 50).astype(np.float32)

# Predict
samples = amortizer.sample({'sim_data': test_input}, n_samples=100)
print("Sample shape:", samples.shape)


TypeError: Inputs to a layer should be tensors. Got 'None' (of type <class 'NoneType'>) as input for layer 'summary_net'.